How do participant’s reports of connection changed when long gaps naturally occurred in their conversations? 

We compute average connection ratings for every instance of a long gap across all the conversations. As a comparison, we compute average connection ratings computed in 2-second increments for the 6 seconds immediately preceding the long gap and the 6 seconds immediately following the long gap. 

To make comparisons across conversations, we compute connection difference scores for every time-point by subtracting the connection ratings during the long gap from the connection ratings at each time-point. 

In [1]:
import pandas as pd 
import numpy as np
import os.path
import glob
from scipy import stats
import statsmodels

base_dir = os.path.dirname(os.getcwd())

## Strangers

In [9]:
long_gap_connection = pd.DataFrame()
long_gap_connection = long_gap_connection.fillna(0)
counter = 0

flist = glob.glob(os.path.join(base_dir, 'Analyses', 'turn_taking', 'strangers', '*.csv'))

for file in flist:
    
    convo = file.split('/')[-1].split('.csv')[0]
    id_1 = file.split('/')[-1].split('_')[0]
    id_2 = file.split('_')[-1].split('.csv')[0]
    
    data = pd.read_csv(file)
    
    long_gaps = data.loc[data['gap_length'] > 2000]
    
    if len(long_gaps) > 0:
        
        connection_1 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'strangers', '{}_{}.csv'.format(id_1, id_2)))
        connection_2 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'strangers', '{}_{}.csv'.format(id_2, id_1)))
        
        connection_1['time_msec'] = connection_1['adjustedTime'] * 1000
        connection_2['time_msec'] = connection_2['adjustedTime'] * 1000
        
        for i in list(long_gaps.index):
            
            if (i > 0):
                
                if (data.at[i-1, 'turn_end_msec'] > 6000) & (data.at[i, 'turn_start_msec'] < 594000): # might change and just give NaNs for ones that are too close

                    # the long gap
                    start_of_gap = data.at[i-1, 'turn_end_msec']
                    end_of_gap = data.at[i, 'turn_start_msec']
                    
                    connection_1_long_gap_subset = connection_1.loc[(connection_1['time_msec'] > start_of_gap) & (connection_1['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_1_during_long_gap = np.mean(connection_1_long_gap_subset['Rating'])
                    
                    connection_2_long_gap_subset = connection_2.loc[(connection_2['time_msec'] > start_of_gap) & (connection_2['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_2_during_long_gap = np.mean(connection_2_long_gap_subset['Rating'])

                    # 1 chunk before long gap

                    start_connection_before_1 = start_of_gap - 2000
                    end_connection_before_1 = start_of_gap
                    
                    connection_1_before_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_1) & (connection_1['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_1_before_long_gap_1 = np.mean(connection_1_before_1_subset['Rating'])
                    
                    connection_2_before_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_1) & (connection_2['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_2_before_long_gap_1 = np.mean(connection_2_before_1_subset['Rating'])

                    # 2 chunks before long gap

                    start_connection_before_2 = start_of_gap - 4000 
                    end_connection_before_2 = start_of_gap - 2000
                    
                    connection_1_before_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_2) & (connection_1['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_1_before_long_gap_2 = np.mean(connection_1_before_2_subset['Rating'])
                    
                    connection_2_before_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_2) & (connection_2['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_2_before_long_gap_2 = np.mean(connection_2_before_2_subset['Rating'])

                    # 3 chunks before long gap

                    start_connection_before_3 = start_of_gap - 6000
                    end_connection_before_3 = start_of_gap - 4000

                    connection_1_before_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_3) & (connection_1['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_1_before_long_gap_3 = np.mean(connection_1_before_3_subset['Rating'])
                    
                    connection_2_before_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_3) & (connection_2['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_2_before_long_gap_3 = np.mean(connection_2_before_3_subset['Rating'])
                    

                    # 1 chunk after long gap 

                    start_connection_after_1 = end_of_gap
                    end_connection_after_1 = end_of_gap + 2000
                    
                    connection_1_after_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_1) & (connection_1['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_1_after_long_gap_1 = np.mean(connection_1_after_1_subset['Rating'])
                    
                    connection_2_after_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_1) & (connection_2['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_2_after_long_gap_1 = np.mean(connection_2_after_1_subset['Rating'])

                    # 2 chunks after long gap 

                    start_connection_after_2 = end_of_gap + 2000 
                    end_connection_after_2 = end_of_gap + 4000
                    
                    connection_1_after_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_2) & (connection_1['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_1_after_long_gap_2 = np.mean(connection_1_after_2_subset['Rating'])
                    
                    connection_2_after_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_2) & (connection_2['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_2_after_long_gap_2 = np.mean(connection_2_after_2_subset['Rating'])

                    # 3 chunks after long gap 

                    start_connection_after_3 = end_of_gap + 4000
                    end_connection_after_3 = end_of_gap + 6000
                    
                    connection_1_after_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_3) & (connection_1['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_1_after_long_gap_3 = np.mean(connection_1_after_3_subset['Rating'])
                    
                    connection_2_after_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_3) & (connection_2['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_2_after_long_gap_3 = np.mean(connection_2_after_3_subset['Rating'])


                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = convo
                    long_gap_connection.at[counter, 'connection_3_before'] = connection_1_before_long_gap_3
                    long_gap_connection.at[counter, 'connection_2_before'] = connection_1_before_long_gap_2
                    long_gap_connection.at[counter, 'connection_1_before'] = connection_1_before_long_gap_1
                    long_gap_connection.at[counter, 'connection_long_gap'] = connection_1_during_long_gap
                    long_gap_connection.at[counter, 'connection_1_after'] = connection_1_after_long_gap_1
                    long_gap_connection.at[counter, 'connection_2_after'] = connection_1_after_long_gap_2
                    long_gap_connection.at[counter, 'connection_3_after'] = connection_1_after_long_gap_3
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    counter += 1

                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = convo
                    long_gap_connection.at[counter, 'connection_3_before'] = connection_2_before_long_gap_3
                    long_gap_connection.at[counter, 'connection_2_before'] = connection_2_before_long_gap_2
                    long_gap_connection.at[counter, 'connection_1_before'] = connection_2_before_long_gap_1
                    long_gap_connection.at[counter, 'connection_long_gap'] = connection_2_during_long_gap
                    long_gap_connection.at[counter, 'connection_1_after'] = connection_2_after_long_gap_1
                    long_gap_connection.at[counter, 'connection_2_after'] = connection_2_after_long_gap_2
                    long_gap_connection.at[counter, 'connection_3_after'] = connection_2_after_long_gap_3
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    counter += 1
        
long_gap_connection['change_in_connection_3_before'] = long_gap_connection['connection_3_before'] - long_gap_connection['connection_long_gap']
long_gap_connection['change_in_connection_2_before'] = long_gap_connection['connection_2_before'] - long_gap_connection['connection_long_gap']
long_gap_connection['change_in_connection_1_before'] = long_gap_connection['connection_1_before'] - long_gap_connection['connection_long_gap']
long_gap_connection['change_in_connection_1_after'] = long_gap_connection['connection_1_after'] - long_gap_connection['connection_long_gap']
long_gap_connection['change_in_connection_2_after'] = long_gap_connection['connection_2_after'] - long_gap_connection['connection_long_gap']
long_gap_connection['change_in_connection_3_after'] = long_gap_connection['connection_3_after'] - long_gap_connection['connection_long_gap']

long_gap_connection.to_csv(os.path.join(base_dir, 'Analyses',
                              'long_gap_connection_strangers.csv'),
                        encoding='utf-8', index=False)


## Friends

In [11]:
long_gap_connection_friends = pd.DataFrame()
long_gap_connection_friends = long_gap_connection_friends.fillna(0)
counter = 0

flist = glob.glob(os.path.join(base_dir, 'Analyses', 'turn_taking', 'friends', '*.csv'))

for file in flist:
    
    convo = file.split('/')[-1].split('.csv')[0]
    id_1 = file.split('/')[-1].split('_')[0]
    id_2 = file.split('_')[-1].split('.csv')[0]
    
    data = pd.read_csv(file)
    
    long_gaps = data.loc[data['gap_length'] > 2000]
    
    if len(long_gaps) > 0:
        
        connection_1 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'friends', '{}_{}.csv'.format(id_1, id_2)))
        connection_2 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'friends', '{}_{}.csv'.format(id_2, id_1)))
        
        connection_1['time_msec'] = connection_1['adjustedTime'] * 1000
        connection_2['time_msec'] = connection_2['adjustedTime'] * 1000
        
        for i in list(long_gaps.index):
            
            if (i > 0):
                
                if (data.at[i-1, 'turn_end_msec'] > 6000) & (data.at[i, 'turn_start_msec'] < 594000): # might change and just give NaNs for ones that are too close

                    # the long gap
                    start_of_gap = data.at[i-1, 'turn_end_msec']
                    end_of_gap = data.at[i, 'turn_start_msec']
                    
                    connection_1_long_gap_subset = connection_1.loc[(connection_1['time_msec'] > start_of_gap) & (connection_1['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_1_during_long_gap = np.mean(connection_1_long_gap_subset['Rating'])
                    
                    connection_2_long_gap_subset = connection_2.loc[(connection_2['time_msec'] > start_of_gap) & (connection_2['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_2_during_long_gap = np.mean(connection_2_long_gap_subset['Rating'])

                    # 1 chunk before long gap

                    start_connection_before_1 = start_of_gap - 2000
                    end_connection_before_1 = start_of_gap
                    
                    connection_1_before_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_1) & (connection_1['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_1_before_long_gap_1 = np.mean(connection_1_before_1_subset['Rating'])
                    
                    connection_2_before_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_1) & (connection_2['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_2_before_long_gap_1 = np.mean(connection_2_before_1_subset['Rating'])

                    # 2 chunks before long gap

                    start_connection_before_2 = start_of_gap - 4000 
                    end_connection_before_2 = start_of_gap - 2000
                    
                    connection_1_before_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_2) & (connection_1['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_1_before_long_gap_2 = np.mean(connection_1_before_2_subset['Rating'])
                    
                    connection_2_before_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_2) & (connection_2['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_2_before_long_gap_2 = np.mean(connection_2_before_2_subset['Rating'])

                    # 3 chunks before long gap

                    start_connection_before_3 = start_of_gap - 6000
                    end_connection_before_3 = start_of_gap - 4000

                    connection_1_before_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_3) & (connection_1['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_1_before_long_gap_3 = np.mean(connection_1_before_3_subset['Rating'])
                    
                    connection_2_before_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_3) & (connection_2['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_2_before_long_gap_3 = np.mean(connection_2_before_3_subset['Rating'])
                    

                    # 1 chunk after long gap 

                    start_connection_after_1 = end_of_gap
                    end_connection_after_1 = end_of_gap + 2000
                    
                    connection_1_after_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_1) & (connection_1['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_1_after_long_gap_1 = np.mean(connection_1_after_1_subset['Rating'])
                    
                    connection_2_after_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_1) & (connection_2['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_2_after_long_gap_1 = np.mean(connection_2_after_1_subset['Rating'])

                    # 2 chunks after long gap 

                    start_connection_after_2 = end_of_gap + 2000 
                    end_connection_after_2 = end_of_gap + 4000
                    
                    connection_1_after_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_2) & (connection_1['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_1_after_long_gap_2 = np.mean(connection_1_after_2_subset['Rating'])
                    
                    connection_2_after_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_2) & (connection_2['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_2_after_long_gap_2 = np.mean(connection_2_after_2_subset['Rating'])

                    # 3 chunks after long gap 

                    start_connection_after_3 = end_of_gap + 4000
                    end_connection_after_3 = end_of_gap + 6000
                    
                    connection_1_after_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_3) & (connection_1['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_1_after_long_gap_3 = np.mean(connection_1_after_3_subset['Rating'])
                    
                    connection_2_after_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_3) & (connection_2['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_2_after_long_gap_3 = np.mean(connection_2_after_3_subset['Rating'])


                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = convo
                    long_gap_connection_friends.at[counter, 'connection_3_before'] = connection_1_before_long_gap_3
                    long_gap_connection_friends.at[counter, 'connection_2_before'] = connection_1_before_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_1_before'] = connection_1_before_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_long_gap'] = connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'connection_1_after'] = connection_1_after_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_2_after'] = connection_1_after_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_3_after'] = connection_1_after_long_gap_3
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    counter += 1

                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = convo
                    long_gap_connection_friends.at[counter, 'connection_3_before'] = connection_2_before_long_gap_3
                    long_gap_connection_friends.at[counter, 'connection_2_before'] = connection_2_before_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_1_before'] = connection_2_before_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_long_gap'] = connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'connection_1_after'] = connection_2_after_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_2_after'] = connection_2_after_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_3_after'] = connection_2_after_long_gap_3
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    counter += 1
        
long_gap_connection_friends['change_in_connection_3_before'] = long_gap_connection_friends['connection_3_before'] - long_gap_connection_friends['connection_long_gap']
long_gap_connection_friends['change_in_connection_2_before'] = long_gap_connection_friends['connection_2_before'] - long_gap_connection_friends['connection_long_gap']
long_gap_connection_friends['change_in_connection_1_before'] = long_gap_connection_friends['connection_1_before'] - long_gap_connection_friends['connection_long_gap']
long_gap_connection_friends['change_in_connection_1_after'] = long_gap_connection_friends['connection_1_after'] - long_gap_connection_friends['connection_long_gap']
long_gap_connection_friends['change_in_connection_2_after'] = long_gap_connection_friends['connection_2_after'] - long_gap_connection_friends['connection_long_gap']
long_gap_connection_friends['change_in_connection_3_after'] = long_gap_connection_friends['connection_3_after'] - long_gap_connection_friends['connection_long_gap']

long_gap_connection_friends.to_csv(os.path.join(base_dir, 'Analyses',
                              'long_gap_connection_friends.csv'),
                        encoding='utf-8', index=False)


## combine stranger and friend data

In [12]:
long_gap_connection['condition'] = 'strangers'
long_gap_connection_friends['condition'] = 'friends'

long_gaps_all = pd.merge(long_gap_connection, long_gap_connection_friends, how='outer')

long_gaps_all.to_csv(os.path.join(base_dir, 'Analyses',
                              'long_gap_connection_all.csv'),
                        encoding='utf-8', index=False)

# correcting for multiple comparisions

Sometime in between first running these analyses and returning to check them before submitting the paper, the statsmodels multitest function stopped working (https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html). I therefore copy and pasted the source code for the function below. Source code came from here: https://www.statsmodels.org/dev/_modules/statsmodels/stats/multitest.html

In [13]:
def _ecdf(x):
    '''no frills empirical cdf used in fdrcorrection
    '''
    nobs = len(x)
    return np.arange(1,nobs+1)/float(nobs)

multitest_methods_names = {'b': 'Bonferroni',
                           's': 'Sidak',
                           'h': 'Holm',
                           'hs': 'Holm-Sidak',
                           'sh': 'Simes-Hochberg',
                           'ho': 'Hommel',
                           'fdr_bh': 'FDR Benjamini-Hochberg',
                           'fdr_by': 'FDR Benjamini-Yekutieli',
                           'fdr_tsbh': 'FDR 2-stage Benjamini-Hochberg',
                           'fdr_tsbky': 'FDR 2-stage Benjamini-Krieger-Yekutieli',
                           'fdr_gbs': 'FDR adaptive Gavrilov-Benjamini-Sarkar'
                           }

_alias_list = [['b', 'bonf', 'bonferroni'],
               ['s', 'sidak'],
               ['h', 'holm'],
               ['hs', 'holm-sidak'],
               ['sh', 'simes-hochberg'],
               ['ho', 'hommel'],
               ['fdr_bh', 'fdr_i', 'fdr_p', 'fdri', 'fdrp'],
               ['fdr_by', 'fdr_n', 'fdr_c', 'fdrn', 'fdrcorr'],
               ['fdr_tsbh', 'fdr_2sbh'],
               ['fdr_tsbky', 'fdr_2sbky', 'fdr_twostage'],
               ['fdr_gbs']
               ]


multitest_alias = {}
for m in _alias_list:
    multitest_alias[m[0]] = m[0]
    for a in m[1:]:
        multitest_alias[a] = m[0]

def multipletests(pvals, alpha=0.05, method='hs', is_sorted=False,
                  returnsorted=False):
    """
    Test results and p-value correction for multiple tests

    Parameters
    ----------
    pvals : array_like, 1-d
        uncorrected p-values.   Must be 1-dimensional.
    alpha : float
        FWER, family-wise error rate, e.g. 0.1
    method : str
        Method used for testing and adjustment of pvalues. Can be either the
        full name or initial letters. Available methods are:

        - `bonferroni` : one-step correction
        - `sidak` : one-step correction
        - `holm-sidak` : step down method using Sidak adjustments
        - `holm` : step-down method using Bonferroni adjustments
        - `simes-hochberg` : step-up method  (independent)
        - `hommel` : closed method based on Simes tests (non-negative)
        - `fdr_bh` : Benjamini/Hochberg  (non-negative)
        - `fdr_by` : Benjamini/Yekutieli (negative)
        - `fdr_tsbh` : two stage fdr correction (non-negative)
        - `fdr_tsbky` : two stage fdr correction (non-negative)

    is_sorted : bool
        If False (default), the p_values will be sorted, but the corrected
        pvalues are in the original order. If True, then it assumed that the
        pvalues are already sorted in ascending order.
    returnsorted : bool
         not tested, return sorted p-values instead of original sequence

    Returns
    -------
    reject : ndarray, boolean
        true for hypothesis that can be rejected for given alpha
    pvals_corrected : ndarray
        p-values corrected for multiple tests
    alphacSidak : float
        corrected alpha for Sidak method
    alphacBonf : float
        corrected alpha for Bonferroni method

    Notes
    -----
    There may be API changes for this function in the future.

    Except for 'fdr_twostage', the p-value correction is independent of the
    alpha specified as argument. In these cases the corrected p-values
    can also be compared with a different alpha. In the case of 'fdr_twostage',
    the corrected p-values are specific to the given alpha, see
    ``fdrcorrection_twostage``.

    The 'fdr_gbs' procedure is not verified against another package, p-values
    are derived from scratch and are not derived in the reference. In Monte
    Carlo experiments the method worked correctly and maintained the false
    discovery rate.

    All procedures that are included, control FWER or FDR in the independent
    case, and most are robust in the positively correlated case.

    `fdr_gbs`: high power, fdr control for independent case and only small
    violation in positively correlated case

    **Timing**:

    Most of the time with large arrays is spent in `argsort`. When
    we want to calculate the p-value for several methods, then it is more
    efficient to presort the pvalues, and put the results back into the
    original order outside of the function.

    Method='hommel' is very slow for large arrays, since it requires the
    evaluation of n partitions, where n is the number of p-values.
    """
    import gc
    pvals = np.asarray(pvals)
    alphaf = alpha  # Notation ?

    if not is_sorted:
        sortind = np.argsort(pvals)
        pvals = np.take(pvals, sortind)

    ntests = len(pvals)
    alphacSidak = 1 - np.power((1. - alphaf), 1./ntests)
    alphacBonf = alphaf / float(ntests)
    if method.lower() in ['b', 'bonf', 'bonferroni']:
        reject = pvals <= alphacBonf
        pvals_corrected = pvals * float(ntests)

    elif method.lower() in ['s', 'sidak']:
        reject = pvals <= alphacSidak
        pvals_corrected = -np.expm1(ntests * np.log1p(-pvals))

    elif method.lower() in ['hs', 'holm-sidak']:
        alphacSidak_all = 1 - np.power((1. - alphaf),
                                       1./np.arange(ntests, 0, -1))
        notreject = pvals > alphacSidak_all
        del alphacSidak_all

        nr_index = np.nonzero(notreject)[0]
        if nr_index.size == 0:
            # nonreject is empty, all rejected
            notrejectmin = len(pvals)
        else:
            notrejectmin = np.min(nr_index)
        notreject[notrejectmin:] = True
        reject = ~notreject
        del notreject

        # It's eqivalent to 1 - np.power((1. - pvals),
        #                           np.arange(ntests, 0, -1))
        # but prevents the issue of the floating point precision
        pvals_corrected_raw = -np.expm1(np.arange(ntests, 0, -1) *
                                        np.log1p(-pvals))
        pvals_corrected = np.maximum.accumulate(pvals_corrected_raw)
        del pvals_corrected_raw

    elif method.lower() in ['h', 'holm']:
        notreject = pvals > alphaf / np.arange(ntests, 0, -1)
        nr_index = np.nonzero(notreject)[0]
        if nr_index.size == 0:
            # nonreject is empty, all rejected
            notrejectmin = len(pvals)
        else:
            notrejectmin = np.min(nr_index)
        notreject[notrejectmin:] = True
        reject = ~notreject
        pvals_corrected_raw = pvals * np.arange(ntests, 0, -1)
        pvals_corrected = np.maximum.accumulate(pvals_corrected_raw)
        del pvals_corrected_raw
        gc.collect()

    elif method.lower() in ['sh', 'simes-hochberg']:
        alphash = alphaf / np.arange(ntests, 0, -1)
        reject = pvals <= alphash
        rejind = np.nonzero(reject)
        if rejind[0].size > 0:
            rejectmax = np.max(np.nonzero(reject))
            reject[:rejectmax] = True
        pvals_corrected_raw = np.arange(ntests, 0, -1) * pvals
        pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
        del pvals_corrected_raw

    elif method.lower() in ['ho', 'hommel']:
        # we need a copy because we overwrite it in a loop
        a = pvals.copy()
        for m in range(ntests, 1, -1):
            cim = np.min(m * pvals[-m:] / np.arange(1,m+1.))
            a[-m:] = np.maximum(a[-m:], cim)
            a[:-m] = np.maximum(a[:-m], np.minimum(m * pvals[:-m], cim))
        pvals_corrected = a
        reject = a <= alphaf

    elif method.lower() in ['fdr_bh', 'fdr_i', 'fdr_p', 'fdri', 'fdrp']:
        # delegate, call with sorted pvals
        reject, pvals_corrected = fdrcorrection(pvals, alpha=alpha,
                                                 method='indep',
                                                 is_sorted=True)
    elif method.lower() in ['fdr_by', 'fdr_n', 'fdr_c', 'fdrn', 'fdrcorr']:
        # delegate, call with sorted pvals
        reject, pvals_corrected = fdrcorrection(pvals, alpha=alpha,
                                                 method='n',
                                                 is_sorted=True)
    elif method.lower() in ['fdr_tsbky', 'fdr_2sbky', 'fdr_twostage']:
        # delegate, call with sorted pvals
        reject, pvals_corrected = fdrcorrection_twostage(pvals, alpha=alpha,
                                                         method='bky',
                                                         is_sorted=True)[:2]
    elif method.lower() in ['fdr_tsbh', 'fdr_2sbh']:
        # delegate, call with sorted pvals
        reject, pvals_corrected = fdrcorrection_twostage(pvals, alpha=alpha,
                                                         method='bh',
                                                         is_sorted=True)[:2]

    elif method.lower() in ['fdr_gbs']:
        #adaptive stepdown in Gavrilov, Benjamini, Sarkar, Annals of Statistics 2009
##        notreject = pvals > alphaf / np.arange(ntests, 0, -1) #alphacSidak
##        notrejectmin = np.min(np.nonzero(notreject))
##        notreject[notrejectmin:] = True
##        reject = ~notreject

        ii = np.arange(1, ntests + 1)
        q = (ntests + 1. - ii)/ii * pvals / (1. - pvals)
        pvals_corrected_raw = np.maximum.accumulate(q) #up requirementd

        pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
        del pvals_corrected_raw
        reject = pvals_corrected <= alpha

    else:
        raise ValueError('method not recognized')

    if pvals_corrected is not None: #not necessary anymore
        pvals_corrected[pvals_corrected>1] = 1
    if is_sorted or returnsorted:
        return reject, pvals_corrected, alphacSidak, alphacBonf
    else:
        pvals_corrected_ = np.empty_like(pvals_corrected)
        pvals_corrected_[sortind] = pvals_corrected
        del pvals_corrected
        reject_ = np.empty_like(reject)
        reject_[sortind] = reject
        return reject_, pvals_corrected_, alphacSidak, alphacBonf



def fdrcorrection(pvals, alpha=0.05, method='indep', is_sorted=False):
    '''
    pvalue correction for false discovery rate.

    This covers Benjamini/Hochberg for independent or positively correlated and
    Benjamini/Yekutieli for general or negatively correlated tests.

    Parameters
    ----------
    pvals : array_like, 1d
        Set of p-values of the individual tests.
    alpha : float, optional
        Family-wise error rate. Defaults to ``0.05``.
    method : {'i', 'indep', 'p', 'poscorr', 'n', 'negcorr'}, optional
        Which method to use for FDR correction.
        ``{'i', 'indep', 'p', 'poscorr'}`` all refer to ``fdr_bh``
        (Benjamini/Hochberg for independent or positively
        correlated tests). ``{'n', 'negcorr'}`` both refer to ``fdr_by``
        (Benjamini/Yekutieli for general or negatively correlated tests).
        Defaults to ``'indep'``.
    is_sorted : bool, optional
        If False (default), the p_values will be sorted, but the corrected
        pvalues are in the original order. If True, then it assumed that the
        pvalues are already sorted in ascending order.

    Returns
    -------
    rejected : ndarray, bool
        True if a hypothesis is rejected, False if not
    pvalue-corrected : ndarray
        pvalues adjusted for multiple hypothesis testing to limit FDR

    Notes
    -----
    If there is prior information on the fraction of true hypothesis, then alpha
    should be set to ``alpha * m/m_0`` where m is the number of tests,
    given by the p-values, and m_0 is an estimate of the true hypothesis.
    (see Benjamini, Krieger and Yekuteli)

    The two-step method of Benjamini, Krieger and Yekutiel that estimates the number
    of false hypotheses will be available (soon).

    Both methods exposed via this function (Benjamini/Hochberg, Benjamini/Yekutieli)
    are also available in the function ``multipletests``, as ``method="fdr_bh"`` and
    ``method="fdr_by"``, respectively.

    See also
    --------
    multipletests

    '''
    pvals = np.asarray(pvals)
    assert pvals.ndim == 1, "pvals must be 1-dimensional, that is of shape (n,)"

    if not is_sorted:
        pvals_sortind = np.argsort(pvals)
        pvals_sorted = np.take(pvals, pvals_sortind)
    else:
        pvals_sorted = pvals  # alias

    if method in ['i', 'indep', 'p', 'poscorr']:
        ecdffactor = _ecdf(pvals_sorted)
    elif method in ['n', 'negcorr']:
        cm = np.sum(1./np.arange(1, len(pvals_sorted)+1))   #corrected this
        ecdffactor = _ecdf(pvals_sorted) / cm
##    elif method in ['n', 'negcorr']:
##        cm = np.sum(np.arange(len(pvals)))
##        ecdffactor = ecdf(pvals_sorted)/cm
    else:
        raise ValueError('only indep and negcorr implemented')
    reject = pvals_sorted <= ecdffactor*alpha
    if reject.any():
        rejectmax = max(np.nonzero(reject)[0])
        reject[:rejectmax] = True

    pvals_corrected_raw = pvals_sorted / ecdffactor
    pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
    del pvals_corrected_raw
    pvals_corrected[pvals_corrected>1] = 1
    if not is_sorted:
        pvals_corrected_ = np.empty_like(pvals_corrected)
        pvals_corrected_[pvals_sortind] = pvals_corrected
        del pvals_corrected
        reject_ = np.empty_like(reject)
        reject_[pvals_sortind] = reject
        return reject_, pvals_corrected_
    else:
        return reject, pvals_corrected



def fdrcorrection_twostage(pvals, alpha=0.05, method='bky', iter=False,
                           is_sorted=False):
    '''(iterated) two stage linear step-up procedure with estimation of number of true
    hypotheses

    Benjamini, Krieger and Yekuteli, procedure in Definition 6

    Parameters
    ----------
    pvals : array_like
        set of p-values of the individual tests.
    alpha : float
        error rate
    method : {'bky', 'bh')
        see Notes for details

        * 'bky' - implements the procedure in Definition 6 of Benjamini, Krieger
           and Yekuteli 2006
        * 'bh' - the two stage method of Benjamini and Hochberg

    iter : bool

    Returns
    -------
    rejected : ndarray, bool
        True if a hypothesis is rejected, False if not
    pvalue-corrected : ndarray
        pvalues adjusted for multiple hypotheses testing to limit FDR
    m0 : int
        ntest - rej, estimated number of true hypotheses
    alpha_stages : list of floats
        A list of alphas that have been used at each stage

    Notes
    -----
    The returned corrected p-values are specific to the given alpha, they
    cannot be used for a different alpha.

    The returned corrected p-values are from the last stage of the fdr_bh
    linear step-up procedure (fdrcorrection0 with method='indep') corrected
    for the estimated fraction of true hypotheses.
    This means that the rejection decision can be obtained with
    ``pval_corrected <= alpha``, where ``alpha`` is the original significance
    level.
    (Note: This has changed from earlier versions (<0.5.0) of statsmodels.)

    BKY described several other multi-stage methods, which would be easy to implement.
    However, in their simulation the simple two-stage method (with iter=False) was the
    most robust to the presence of positive correlation

    TODO: What should be returned?

    '''
    pvals = np.asarray(pvals)

    if not is_sorted:
        pvals_sortind = np.argsort(pvals)
        pvals = np.take(pvals, pvals_sortind)

    ntests = len(pvals)
    if method == 'bky':
        fact = (1.+alpha)
        alpha_prime = alpha / fact
    elif method == 'bh':
        fact = 1.
        alpha_prime = alpha
    else:
        raise ValueError("only 'bky' and 'bh' are available as method")

    alpha_stages = [alpha_prime]
    rej, pvalscorr = fdrcorrection(pvals, alpha=alpha_prime, method='indep',
                                   is_sorted=True)
    r1 = rej.sum()
    if (r1 == 0) or (r1 == ntests):
        return rej, pvalscorr * fact, ntests - r1, alpha_stages
    ri_old = r1

    while True:
        ntests0 = 1.0 * ntests - ri_old
        alpha_star = alpha_prime * ntests / ntests0
        alpha_stages.append(alpha_star)
        #print ntests0, alpha_star
        rej, pvalscorr = fdrcorrection(pvals, alpha=alpha_star, method='indep',
                                       is_sorted=True)
        ri = rej.sum()
        if (not iter) or ri == ri_old:
            break
        elif ri < ri_old:
            # prevent cycles and endless loops
            raise RuntimeError(" oops - should not be here")
        ri_old = ri

    # make adjustment to pvalscorr to reflect estimated number of Non-Null cases
    # decision is then pvalscorr < alpha  (or <=)
    pvalscorr *= ntests0 * 1.0 /  ntests
    if method == 'bky':
        pvalscorr *= (1. + alpha)

    if not is_sorted:
        pvalscorr_ = np.empty_like(pvalscorr)
        pvalscorr_[pvals_sortind] = pvalscorr
        del pvalscorr
        reject = np.empty_like(rej)
        reject[pvals_sortind] = rej
        return reject, pvalscorr_, ntests - ri, alpha_stages
    else:
        return rej, pvalscorr, ntests - ri, alpha_stages

## stats -- within convo: strangers

In [14]:
pvals = []

In [15]:
first = stats.ttest_1samp(long_gap_connection['change_in_connection_3_before'].dropna(), 0) # not sig
print(first)
print('df:' + str(len(long_gap_connection['change_in_connection_3_before'].dropna())-1))
pvals.append(first[1])

Ttest_1sampResult(statistic=0.7112813876828835, pvalue=0.4771631015885298)
df:655


In [16]:
second = stats.ttest_1samp(long_gap_connection['change_in_connection_2_before'].dropna(), 0) # not sig
print(second)
print('df:' + str(len(long_gap_connection['change_in_connection_2_before'].dropna())-1))
pvals.append(second[1])

Ttest_1sampResult(statistic=1.8164183514423367, pvalue=0.06976319071928287)
df:655


In [17]:
third = stats.ttest_1samp(long_gap_connection['change_in_connection_1_before'].dropna(), 0)
print(third)
print('df:' + str(len(long_gap_connection['change_in_connection_1_before'].dropna())-1))
pvals.append(third[1])

Ttest_1sampResult(statistic=2.45281395795911, pvalue=0.014434035309558887)
df:655


In [18]:
fourth = stats.ttest_1samp(long_gap_connection['change_in_connection_1_after'].dropna(), 0) 
print(fourth)
print('df:' + str(len(long_gap_connection['change_in_connection_1_after'].dropna())-1))
pvals.append(fourth[1])

Ttest_1sampResult(statistic=-3.422122610641923, pvalue=0.0006600081343947048)
df:655


In [19]:
fifth = stats.ttest_1samp(long_gap_connection['change_in_connection_2_after'].dropna(), 0)
print(fifth)
print('df:' + str(len(long_gap_connection['change_in_connection_2_after'].dropna())-1))
pvals.append(fifth[1])

Ttest_1sampResult(statistic=-2.1745623441672635, pvalue=0.030020105042069565)
df:655


In [20]:
sixth = stats.ttest_1samp(long_gap_connection['change_in_connection_3_after'].dropna(), 0)  # not sig
print(sixth)
print('df:' + str(len(long_gap_connection['change_in_connection_3_after'].dropna())-1))
pvals.append(sixth[1])

Ttest_1sampResult(statistic=-1.485605118746222, pvalue=0.13786472575297623)
df:655


In [21]:
#statsmodels.stats.multitest.multipletests(pvals, alpha=0.05, method='fdr_tsbky', is_sorted=False, returnsorted=False)
# adjust p-values for multiple comparisions

multipletests(pvals, alpha=0.05, method='fdr_tsbky', is_sorted=False, returnsorted=False)

(array([False, False,  True,  True,  True, False]),
 array([0.33401417, 0.07325135, 0.03031147, 0.00277203, 0.04202815,
        0.11580637]),
 0.008512444610847103,
 0.008333333333333333)

## stats -- within convo: friends

In [22]:
pvals = []

In [23]:
first = stats.ttest_1samp(long_gap_connection_friends['change_in_connection_3_before'].dropna(), 0)
print(first)
print('df:' + str(len(long_gap_connection_friends['change_in_connection_3_before'].dropna())-1))
pvals.append(first[1])


Ttest_1sampResult(statistic=-3.4391851938662317, pvalue=0.000644333906067079)
df:403


In [24]:
second = stats.ttest_1samp(long_gap_connection_friends['change_in_connection_2_before'].dropna(), 0)
print(second)
print('df:' + str(len(long_gap_connection_friends['change_in_connection_2_before'].dropna())-1))
pvals.append(second[1])


Ttest_1sampResult(statistic=-3.390324204538273, pvalue=0.0007670103842778143)
df:403


In [25]:
third = stats.ttest_1samp(long_gap_connection_friends['change_in_connection_1_before'].dropna(), 0)
print(third)
print('df:' + str(len(long_gap_connection_friends['change_in_connection_1_before'].dropna())-1))
pvals.append(third[1])


Ttest_1sampResult(statistic=-2.204406394522009, pvalue=0.028060279059486454)
df:403


In [26]:
fourth = stats.ttest_1samp(long_gap_connection_friends['change_in_connection_1_after'].dropna(), 0) # not sig
print(fourth)
print('df:' + str(len(long_gap_connection_friends['change_in_connection_1_after'].dropna())-1))
pvals.append(fourth[1])


Ttest_1sampResult(statistic=-1.289175349303128, pvalue=0.19807658995481292)
df:403


In [27]:
fifth = stats.ttest_1samp(long_gap_connection_friends['change_in_connection_2_after'].dropna(), 0) # not sig
print(fifth)
print('df:' + str(len(long_gap_connection_friends['change_in_connection_2_after'].dropna())-1))
pvals.append(fifth[1])

Ttest_1sampResult(statistic=-1.541104955960675, pvalue=0.1240760578888544)
df:403


In [28]:
sixth = stats.ttest_1samp(long_gap_connection_friends['change_in_connection_3_after'].dropna(), 0) # not sig
print(sixth)
print('df:' + str(len(long_gap_connection_friends['change_in_connection_3_after'].dropna())-1))
pvals.append(sixth[1])

Ttest_1sampResult(statistic=-1.726476765154216, pvalue=0.08502803635775963)
df:403


In [29]:
#statsmodels.stats.multitest.multipletests(pvals, alpha=0.05, method='fdr_tsbky', is_sorted=False, returnsorted=False)
# adjust p-values for multiple comparisions

multipletests(pvals, alpha=0.05, method='fdr_tsbky', is_sorted=False, returnsorted=False)

(array([ True,  True,  True, False, False, False]),
 array([0.00161072, 0.00161072, 0.03928439, 0.13865361, 0.10422389,
        0.08927944]),
 0.008512444610847103,
 0.008333333333333333)

## stats -- between groups

In [30]:
pvals = []

In [31]:
first = stats.ttest_ind(long_gap_connection['change_in_connection_3_before'].dropna(), 
                long_gap_connection_friends['change_in_connection_3_before'].dropna(), 
                equal_var=False)
print(first)
print('df:' + str((len(long_gap_connection['change_in_connection_3_before'].dropna()))+len(long_gap_connection_friends['change_in_connection_3_before'].dropna())-2))
pvals.append(first[1])

Ttest_indResult(statistic=2.8933453972411227, pvalue=0.003892976144933679)
df:1058


In [32]:
second = stats.ttest_ind(long_gap_connection['change_in_connection_2_before'].dropna(), 
                long_gap_connection_friends['change_in_connection_2_before'].dropna(), 
                equal_var=False)
print(second)
print('df:' + str((len(long_gap_connection['change_in_connection_2_before'].dropna()))+len(long_gap_connection_friends['change_in_connection_2_before'].dropna())-2))
pvals.append(second[1])

Ttest_indResult(statistic=3.718819831849168, pvalue=0.00021174307311687412)
df:1058


In [33]:
third = stats.ttest_ind(long_gap_connection['change_in_connection_1_before'].dropna(), 
                long_gap_connection_friends['change_in_connection_1_before'].dropna(), 
                equal_var=False)
print(third)
print('df:' + str((len(long_gap_connection['change_in_connection_1_before'].dropna()))+len(long_gap_connection_friends['change_in_connection_1_before'].dropna())-2))
pvals.append(third[1])

Ttest_indResult(statistic=3.284849749162399, pvalue=0.0010572542304095652)
df:1058


In [34]:
fourth = stats.ttest_ind(long_gap_connection['change_in_connection_1_after'].dropna(), 
                long_gap_connection_friends['change_in_connection_1_after'].dropna(), 
                equal_var=False)
print(fourth)
print('df:' + str((len(long_gap_connection['change_in_connection_1_after'].dropna()))+len(long_gap_connection_friends['change_in_connection_1_after'].dropna())-2))
pvals.append(fourth[1])


Ttest_indResult(statistic=-0.23761030902318284, pvalue=0.8122686542506321)
df:1058


In [35]:
fifth = stats.ttest_ind(long_gap_connection['change_in_connection_2_after'].dropna(), 
                long_gap_connection_friends['change_in_connection_2_after'].dropna(), 
                equal_var=False)
print(fifth)
print('df:' + str((len(long_gap_connection['change_in_connection_2_after'].dropna()))+len(long_gap_connection_friends['change_in_connection_2_after'].dropna())-2))
pvals.append(fifth[1])

Ttest_indResult(statistic=0.24934347000150373, pvalue=0.8031715596488865)
df:1058


In [36]:
sixth = stats.ttest_ind(long_gap_connection['change_in_connection_3_after'].dropna(), 
                long_gap_connection_friends['change_in_connection_3_after'].dropna(), 
                equal_var=False)
print(sixth)
print('df:' + str((len(long_gap_connection['change_in_connection_3_after'].dropna()))+len(long_gap_connection_friends['change_in_connection_3_after'].dropna())-2))
pvals.append(sixth[1])

Ttest_indResult(statistic=0.66735379988914, pvalue=0.5047606588354114)
df:1058


In [37]:
#statsmodels.stats.multitest.multipletests(pvals, alpha=0.05, method='fdr_tsbky', is_sorted=False, returnsorted=False)
# adjust p-values for multiple comparisions

multipletests(pvals, alpha=0.05, method='fdr_tsbky', is_sorted=False, returnsorted=False)

(array([ True,  True,  True, False, False, False]),
 array([0.00408762, 0.00066699, 0.00166518, 0.42644104, 0.42644104,
        0.39749902]),
 0.008512444610847103,
 0.008333333333333333)

# Output a long-form version for plotting purposes

In [42]:
long_gap_connection = pd.DataFrame()
long_gap_connection = long_gap_connection.fillna(0)
counter = 0

flist = glob.glob(os.path.join(base_dir, 'Analyses', 'turn_taking', 'strangers', '*.csv'))

for file in flist:
    
    id_1 = file.split('/')[-1].split('_')[0]
    id_2 = file.split('_')[-1].split('.csv')[0]
    
    data = pd.read_csv(file)
    
    long_gaps = data.loc[data['gap_length'] > 2000]
    
    if len(long_gaps) > 0:
        
        connection_1 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'strangers', '{}_{}.csv'.format(id_1, id_2)))
        connection_2 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'strangers', '{}_{}.csv'.format(id_2, id_1)))
        
        connection_1['time_msec'] = connection_1['adjustedTime'] * 1000
        connection_2['time_msec'] = connection_2['adjustedTime'] * 1000
        
        for i in list(long_gaps.index): 
            
            if (i > 0):
                
                if (data.at[i-1, 'turn_end_msec'] > 6000) & (data.at[i, 'turn_start_msec'] < 594000): # might change and just give NaNs for ones that are too close

                    # the long gap
                    
                    start_of_gap = data.at[i-1, 'turn_end_msec']
                    end_of_gap = data.at[i, 'turn_start_msec']
                    
                    connection_1_long_gap_subset = connection_1.loc[(connection_1['time_msec'] > start_of_gap) & (connection_1['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_1_during_long_gap = np.mean(connection_1_long_gap_subset['Rating'])
                    
                    connection_2_long_gap_subset = connection_2.loc[(connection_2['time_msec'] > start_of_gap) & (connection_2['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_2_during_long_gap = np.mean(connection_2_long_gap_subset['Rating'])
                    
                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = 'long_gap'
                    long_gap_connection.at[counter, 'connection'] = connection_1_during_long_gap
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_during_long_gap - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = 'long_gap'
                    long_gap_connection.at[counter, 'connection'] = connection_2_during_long_gap
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_during_long_gap - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    counter += 1
                    
                    # 1 chunk before long gap

                    start_connection_before_1 = start_of_gap - 2000
                    end_connection_before_1 = start_of_gap
                    
                    connection_1_before_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_1) & (connection_1['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_1_before_long_gap_1 = np.mean(connection_1_before_1_subset['Rating'])
                    
                    connection_2_before_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_1) & (connection_2['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_2_before_long_gap_1 = np.mean(connection_2_before_1_subset['Rating'])
                    
                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '1_before'
                    long_gap_connection.at[counter, 'connection'] = connection_1_before_long_gap_1
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_before_long_gap_1 - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '1_before'
                    long_gap_connection.at[counter, 'connection'] = connection_2_before_long_gap_1
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_before_long_gap_1 - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1

                    # 2 chunks before long gap

                    start_connection_before_2 = start_of_gap - 4000 
                    end_connection_before_2 = start_of_gap - 2000
                    
                    connection_1_before_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_2) & (connection_1['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_1_before_long_gap_2 = np.mean(connection_1_before_2_subset['Rating'])
                    
                    connection_2_before_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_2) & (connection_2['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_2_before_long_gap_2 = np.mean(connection_2_before_2_subset['Rating'])
                    
                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '2_before'
                    long_gap_connection.at[counter, 'connection'] = connection_1_before_long_gap_2
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_before_long_gap_2 - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '2_before'
                    long_gap_connection.at[counter, 'connection'] = connection_2_before_long_gap_2
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_before_long_gap_2 - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1

                    # 3 chunks before long gap

                    start_connection_before_3 = start_of_gap - 6000
                    end_connection_before_3 = start_of_gap - 4000

                    connection_1_before_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_3) & (connection_1['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_1_before_long_gap_3 = np.mean(connection_1_before_3_subset['Rating'])
                    
                    connection_2_before_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_3) & (connection_2['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_2_before_long_gap_3 = np.mean(connection_2_before_3_subset['Rating'])
                    
                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '3_before'
                    long_gap_connection.at[counter, 'connection'] = connection_1_before_long_gap_3
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_before_long_gap_3 - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '3_before'
                    long_gap_connection.at[counter, 'connection'] = connection_2_before_long_gap_3
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_before_long_gap_3 - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1

                    # 1 chunk after long gap 

                    start_connection_after_1 = end_of_gap
                    end_connection_after_1 = end_of_gap + 2000
                    
                    connection_1_after_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_1) & (connection_1['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_1_after_long_gap_1 = np.mean(connection_1_after_1_subset['Rating'])
                    
                    connection_2_after_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_1) & (connection_2['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_2_after_long_gap_1 = np.mean(connection_2_after_1_subset['Rating'])
                    
                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '1_after'
                    long_gap_connection.at[counter, 'connection'] = connection_1_after_long_gap_1
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_after_long_gap_1 - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '1_after'
                    long_gap_connection.at[counter, 'connection'] = connection_2_after_long_gap_1
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_after_long_gap_1 - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1

                    # 2 chunks after long gap 

                    start_connection_after_2 = end_of_gap + 2000 
                    end_connection_after_2 = end_of_gap + 4000
                    
                    connection_1_after_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_2) & (connection_1['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_1_after_long_gap_2 = np.mean(connection_1_after_2_subset['Rating'])
                    
                    connection_2_after_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_2) & (connection_2['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_2_after_long_gap_2 = np.mean(connection_2_after_2_subset['Rating'])
                    
                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '2_after'
                    long_gap_connection.at[counter, 'connection'] = connection_1_after_long_gap_2
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_after_long_gap_2 - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '2_after'
                    long_gap_connection.at[counter, 'connection'] = connection_2_after_long_gap_2
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_after_long_gap_2 - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1

                    # 3 chunks after long gap 

                    start_connection_after_3 = end_of_gap + 4000
                    end_connection_after_3 = end_of_gap + 6000
                    
                    connection_1_after_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_3) & (connection_1['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_1_after_long_gap_3 = np.mean(connection_1_after_3_subset['Rating'])
                    
                    connection_2_after_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_3) & (connection_2['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_2_after_long_gap_3 = np.mean(connection_2_after_3_subset['Rating'])

                    long_gap_connection.at[counter, 'subID'] = id_1
                    long_gap_connection.at[counter, 'partnerID'] = id_2
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '3_after'
                    long_gap_connection.at[counter, 'connection'] = connection_1_after_long_gap_3
                    long_gap_connection.at[counter, 'connection_change'] = connection_1_after_long_gap_3 - connection_1_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection.at[counter, 'subID'] = id_2
                    long_gap_connection.at[counter, 'partnerID'] = id_1
                    long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection.at[counter, 'timepoint'] = '3_after'
                    long_gap_connection.at[counter, 'connection'] = connection_2_after_long_gap_3
                    long_gap_connection.at[counter, 'connection_change'] = connection_2_after_long_gap_3 - connection_2_during_long_gap
                    long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    

In [43]:
long_gap_connection_friends = pd.DataFrame()
long_gap_connection_friends = long_gap_connection_friends.fillna(0)
counter = 0

flist = glob.glob(os.path.join(base_dir, 'Analyses', 'turn_taking', 'friends', '*.csv'))

for file in flist:
    
    id_1 = file.split('/')[-1].split('_')[0]
    id_2 = file.split('_')[-1].split('.csv')[0]
    
    data = pd.read_csv(file)
    
    long_gaps = data.loc[data['gap_length'] > 2000]
    
    if len(long_gaps) > 0:
        
        connection_1 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'friends', '{}_{}.csv'.format(id_1, id_2)))
        connection_2 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'friends', '{}_{}.csv'.format(id_2, id_1)))
        
        connection_1['time_msec'] = connection_1['adjustedTime'] * 1000
        connection_2['time_msec'] = connection_2['adjustedTime'] * 1000
        
        for i in list(long_gaps.index): 
            
            if (i > 0):
                
                if (data.at[i-1, 'turn_end_msec'] > 6000) & (data.at[i, 'turn_start_msec'] < 594000): # might change and just give NaNs for ones that are too close

                    # the long gap
                    
                    start_of_gap = data.at[i-1, 'turn_end_msec']
                    end_of_gap = data.at[i, 'turn_start_msec']
                    
                    connection_1_long_gap_subset = connection_1.loc[(connection_1['time_msec'] > start_of_gap) & (connection_1['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_1_during_long_gap = np.mean(connection_1_long_gap_subset['Rating'])
                    
                    connection_2_long_gap_subset = connection_2.loc[(connection_2['time_msec'] > start_of_gap) & (connection_2['time_msec'] < end_of_gap)].reset_index(drop=True)
                    connection_2_during_long_gap = np.mean(connection_2_long_gap_subset['Rating'])
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = 'long_gap'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_during_long_gap - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = 'long_gap'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_during_long_gap - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']
                    
                    counter += 1

                    # 1 chunk before long gap

                    start_connection_before_1 = start_of_gap - 2000
                    end_connection_before_1 = start_of_gap
                    
                    connection_1_before_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_1) & (connection_1['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_1_before_long_gap_1 = np.mean(connection_1_before_1_subset['Rating'])
                    
                    connection_2_before_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_1) & (connection_2['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                    connection_2_before_long_gap_1 = np.mean(connection_2_before_1_subset['Rating'])
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '1_before'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_before_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_before_long_gap_1 - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '1_before'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_before_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_before_long_gap_1 - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1

                    # 2 chunks before long gap

                    start_connection_before_2 = start_of_gap - 4000 
                    end_connection_before_2 = start_of_gap - 2000
                    
                    connection_1_before_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_2) & (connection_1['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_1_before_long_gap_2 = np.mean(connection_1_before_2_subset['Rating'])
                    
                    connection_2_before_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_2) & (connection_2['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                    connection_2_before_long_gap_2 = np.mean(connection_2_before_2_subset['Rating'])
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '2_before'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_before_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_before_long_gap_2 - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '2_before'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_before_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_before_long_gap_2 - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1

                    # 3 chunks before long gap

                    start_connection_before_3 = start_of_gap - 6000
                    end_connection_before_3 = start_of_gap - 4000

                    connection_1_before_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_3) & (connection_1['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_1_before_long_gap_3 = np.mean(connection_1_before_3_subset['Rating'])
                    
                    connection_2_before_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_3) & (connection_2['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                    connection_2_before_long_gap_3 = np.mean(connection_2_before_3_subset['Rating'])
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '3_before'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_before_long_gap_3
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_before_long_gap_3 - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '3_before'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_before_long_gap_3
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_before_long_gap_3 - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1

                    # 1 chunk after long gap 

                    start_connection_after_1 = end_of_gap
                    end_connection_after_1 = end_of_gap + 2000
                    
                    connection_1_after_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_1) & (connection_1['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_1_after_long_gap_1 = np.mean(connection_1_after_1_subset['Rating'])
                    
                    connection_2_after_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_1) & (connection_2['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                    connection_2_after_long_gap_1 = np.mean(connection_2_after_1_subset['Rating'])
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '1_after'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_after_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_after_long_gap_1 - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '1_after'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_after_long_gap_1
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_after_long_gap_1 - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1

                    # 2 chunks after long gap 

                    start_connection_after_2 = end_of_gap + 2000 
                    end_connection_after_2 = end_of_gap + 4000
                    
                    connection_1_after_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_2) & (connection_1['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_1_after_long_gap_2 = np.mean(connection_1_after_2_subset['Rating'])
                    
                    connection_2_after_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_2) & (connection_2['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                    connection_2_after_long_gap_2 = np.mean(connection_2_after_2_subset['Rating'])
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '2_after'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_after_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_after_long_gap_2 - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '2_after'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_after_long_gap_2
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_after_long_gap_2 - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1

                    # 3 chunks after long gap 

                    start_connection_after_3 = end_of_gap + 4000
                    end_connection_after_3 = end_of_gap + 6000
                    
                    connection_1_after_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_3) & (connection_1['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_1_after_long_gap_3 = np.mean(connection_1_after_3_subset['Rating'])
                    
                    connection_2_after_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_3) & (connection_2['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                    connection_2_after_long_gap_3 = np.mean(connection_2_after_3_subset['Rating'])

                    long_gap_connection_friends.at[counter, 'subID'] = id_1
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '3_after'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_1_after_long_gap_3
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_after_long_gap_3 - connection_1_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    
                    long_gap_connection_friends.at[counter, 'subID'] = id_2
                    long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                    long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                    long_gap_connection_friends.at[counter, 'timepoint'] = '3_after'
                    long_gap_connection_friends.at[counter, 'connection'] = connection_2_after_long_gap_3
                    long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_after_long_gap_3 - connection_2_during_long_gap
                    long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                    
                    counter += 1
                    

In [44]:
long_gap_connection['condition'] = 'strangers'
long_gap_connection_friends['condition'] = 'friends'

long_gaps_all = pd.merge(long_gap_connection, long_gap_connection_friends, how='outer')

long_gaps_all.to_csv(os.path.join(base_dir, 'Analyses',
                              'long_gap_connection_all_long_format.csv'),
                        encoding='utf-8', index=False)